# Nivell 1

- Exercici 1

Agafa el conjunt de dades que vulguis i realitza un pipeline i un gridsearch aplicant l'algorisme de Random Forest.

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin

In [4]:
delayed_flights = pd.read_csv('C:\\Users\\formacio\\Gabriel De Santis\\Datasets\\DelayedFlights.csv', sep=',')
delayed_flights

,Unnamed: 0,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,0,2008,1,3,4,2003.0,1955,2211.0,2225,WN,...,4.0,8.0,0,N,0,NaN,NaN,NaN,NaN,NaN
1,1,2008,1,3,4,754.0,735,1002.0,1000,WN,...,5.0,10.0,0,N,0,NaN,NaN,NaN,NaN,NaN
2,2,2008,1,3,4,628.0,620,804.0,750,WN,...,3.0,17.0,0,N,0,NaN,NaN,NaN,NaN,NaN
3,4,2008,1,3,4,1829.0,1755,1959.0,1925,WN,...,3.0,10.0,0,N,0,2.0,0.0,0.0,0.0,32.0
4,5,2008,1,3,4,1940.0,1915,2121.0,2110,WN,...,4.0,10.0,0,N,0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1936753,7009710,2008,12,13,6,1250.0,1220,1617.0,1552,DL,...,9.0,18.0,0,N,0,3.0,0.0,0.0,0.0,22.0
1936754,7009717,2008,12,13,6,657.0,600,904.0,749,DL,...,15.0,34.0,0,N,0,0.0,57.0,18.0,0.0,0.0
1936755,7009718,2008,12,13,6,1007.0,847,1149.0,1010,DL,...,8.0,32.0,0,N,0,1.0,0.0,19.0,0.0,79.0
1936756,7009726,2008,12,13,6,1251.0,1240,1446.0,1437,DL,...,13.0,13.0,0,N,0,NaN,NaN,NaN,NaN,NaN


In [5]:
delayed_flights.drop(columns='Unnamed: 0', inplace=True)

En exercicis anteriors vam observar que les variables tipus month, week, etc. corresponíen a criteris cíclics, que els valors de UniqueCarrier, FlightNum, etc. eres categoríes que podíem codificar com a valors nominals i que la resta de valors eren numèrics. Aplicarem, per tant, tres encodings diferents segons la variable al nostre pipeline.

In [6]:
cyclical = ['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'CRSDepTime', 'ArrTime', 'CRSArrTime']
nominal = ['UniqueCarrier', 'FlightNum', 'TailNum', 'Origin', 'Dest', 'CancellationCode']
numerical = ['ActualElapsedTime', 'CRSElapsedTime', 'AirTime', 
             'DepDelay', 'Distance', 'TaxiIn', 'TaxiOut', 'CarrierDelay',
           'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']

In [7]:
delayed_flights[cyclical]

,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime
0,1,3,4,2003.0,1955,2211.0,2225
1,1,3,4,754.0,735,1002.0,1000
2,1,3,4,628.0,620,804.0,750
3,1,3,4,1829.0,1755,1959.0,1925
4,1,3,4,1940.0,1915,2121.0,2110
...,...,...,...,...,...,...,...
1936753,12,13,6,1250.0,1220,1617.0,1552
1936754,12,13,6,657.0,600,904.0,749
1936755,12,13,6,1007.0,847,1149.0,1010
1936756,12,13,6,1251.0,1240,1446.0,1437


In [8]:
delayed_flights[nominal]

,UniqueCarrier,FlightNum,TailNum,Origin,Dest,CancellationCode
0,WN,335,N712SW,IAD,TPA,N
1,WN,3231,N772SW,IAD,TPA,N
2,WN,448,N428WN,IND,BWI,N
3,WN,3920,N464WN,IND,BWI,N
4,WN,378,N726SW,IND,JAX,N
...,...,...,...,...,...,...
1936753,DL,1621,N938DL,MSP,ATL,N
1936754,DL,1631,N3743H,RIC,ATL,N
1936755,DL,1631,N909DA,ATL,IAH,N
1936756,DL,1639,N646DL,IAD,ATL,N


In [9]:
delayed_flights[numerical]

,ActualElapsedTime,CRSElapsedTime,AirTime,DepDelay,Distance,TaxiIn,TaxiOut,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
0,128.0,150.0,116.0,8.0,810,4.0,8.0,NaN,NaN,NaN,NaN,NaN
1,128.0,145.0,113.0,19.0,810,5.0,10.0,NaN,NaN,NaN,NaN,NaN
2,96.0,90.0,76.0,8.0,515,3.0,17.0,NaN,NaN,NaN,NaN,NaN
3,90.0,90.0,77.0,34.0,515,3.0,10.0,2.0,0.0,0.0,0.0,32.0
4,101.0,115.0,87.0,25.0,688,4.0,10.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1936753,147.0,152.0,120.0,30.0,906,9.0,18.0,3.0,0.0,0.0,0.0,22.0
1936754,127.0,109.0,78.0,57.0,481,15.0,34.0,0.0,57.0,18.0,0.0,0.0
1936755,162.0,143.0,122.0,80.0,689,8.0,32.0,1.0,0.0,19.0,0.0,79.0
1936756,115.0,117.0,89.0,11.0,533,13.0,13.0,NaN,NaN,NaN,NaN,NaN


In [10]:
delayed_flights.isna().sum() / len(delayed_flights) * 100

Year                  0.000000
Month                 0.000000
DayofMonth            0.000000
DayOfWeek             0.000000
DepTime               0.000000
CRSDepTime            0.000000
ArrTime               0.367108
CRSArrTime            0.000000
UniqueCarrier         0.000000
FlightNum             0.000000
TailNum               0.000258
ActualElapsedTime     0.433043
CRSElapsedTime        0.010223
AirTime               0.433043
ArrDelay              0.433043
DepDelay              0.000000
Origin                0.000000
Dest                  0.000000
Distance              0.000000
TaxiIn                0.367108
TaxiOut               0.023493
Cancelled             0.000000
CancellationCode      0.000000
Diverted              0.000000
CarrierDelay         35.588855
WeatherDelay         35.588855
NASDelay             35.588855
SecurityDelay        35.588855
LateAircraftDelay    35.588855
dtype: float64

Observem que els vols cancelats produeixen una gran quantitat de null values a diferents variables. Pensem que lo millor serà eliminar les files, ja que el nombre de vols cancelats és petit.

In [11]:
delayed_flights[delayed_flights.Cancelled == 1].isna().sum() / len(delayed_flights[delayed_flights.Cancelled == 1]) * 100

Year                   0.000000
Month                  0.000000
DayofMonth             0.000000
DayOfWeek              0.000000
DepTime                0.000000
CRSDepTime             0.000000
ArrTime              100.000000
CRSArrTime             0.000000
UniqueCarrier          0.000000
FlightNum              0.000000
TailNum                0.000000
ActualElapsedTime    100.000000
CRSElapsedTime         0.000000
AirTime              100.000000
ArrDelay             100.000000
DepDelay               0.000000
Origin                 0.000000
Dest                   0.000000
Distance               0.000000
TaxiIn               100.000000
TaxiOut               71.879937
Cancelled              0.000000
CancellationCode       0.000000
Diverted               0.000000
CarrierDelay         100.000000
WeatherDelay         100.000000
NASDelay             100.000000
SecurityDelay        100.000000
LateAircraftDelay    100.000000
dtype: float64

Sembla que els null values de les variables CarrierDelay, WeatherDelay, etc. están relacionats amb ArrDelay, és a dir, que quan el retràs no supera els 14 minuts, no es recullen valors. En aquest cas, creiem que serà convenient imputar-los per 0.

In [12]:
delayed_flights[delayed_flights.CarrierDelay.isna()].ArrDelay.max()

14.0

Sembla que els vols desviats també afecten als valors nuls d'altres variables. Així que els eliminarem de la mateixa forma que els cancelats.

In [13]:
delayed_flights[delayed_flights.Diverted == 1].isna().sum() / len(delayed_flights[delayed_flights.Diverted == 1]) * 100

Year                   0.000000
Month                  0.000000
DayofMonth             0.000000
DayOfWeek              0.000000
DepTime                0.000000
CRSDepTime             0.000000
ArrTime               83.531081
CRSArrTime             0.000000
UniqueCarrier          0.000000
FlightNum              0.000000
TailNum                0.025793
ActualElapsedTime    100.000000
CRSElapsedTime         2.553521
AirTime              100.000000
ArrDelay             100.000000
DepDelay               0.000000
Origin                 0.000000
Dest                   0.000000
Distance               0.000000
TaxiIn                83.531081
TaxiOut                0.000000
Cancelled              0.000000
CancellationCode       0.000000
Diverted               0.000000
CarrierDelay         100.000000
WeatherDelay         100.000000
NASDelay             100.000000
SecurityDelay        100.000000
LateAircraftDelay    100.000000
dtype: float64

hem decidit que imputarem els valors nuls de TailNum per la moda.

In [14]:
delayed_flights.TailNum.isna().sum() / len(delayed_flights) * 100

0.0002581633843773977

Crearem els estimators per introduir-los al nostre pipeline més endavant.

In [153]:
class DataframeFunctionTransformer():
    def __init__(self, func):
        self.func = func

    def transform(self, input_df, **transform_params):
        return self.func(input_df)

    def fit(self, X, y=None, **fit_params):
        return self

In [152]:
def Preprocessing(X):

    #ELIMINAR VALORES NULOS
    X = X.drop(columns='Year') #Delete Year
    X = X.drop(index=X[X.Cancelled == 1].index.values) #Delete Null values debido a Cancelled
    X = X.drop(columns='Cancelled')
    X = X.reset_index(drop=True)
    X = X.drop(index=X[X.Diverted == 1].index.values) #Delete null values debido Diverted
    X = X.drop(columns='Diverted')
    X = X.reset_index(drop=True)
    X.TailNum = X.TailNum.fillna(X.TailNum.mode()[0])
    X[['CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']] = X[['CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']].fillna(0)
    
    #TRANSFORMACION A VARIABLES CÍCLICAS
    X['mnth_sin'] = np.sin((X.Month -1)*(2.*np.pi/12))
    X['mnth_cos'] = np.cos((X.Month -1)*(2.*np.pi/12))
    X['day_M_sin'] = np.sin((X.DayofMonth -1)*(2.*np.pi/31))
    X['day_M_cos'] = np.cos((X.DayofMonth -1)*(2.*np.pi/31))
    X['day_W_sin'] = np.sin((X.DayOfWeek -1)*(2.*np.pi/7))
    X['day_W_cos'] = np.cos((X.DayOfWeek -1)*(2.*np.pi/7))
    X['DepTime_sin'] = np.sin((X.DepTime)*(2.*np.pi/2400))
    X['DepTime_cos'] = np.cos((X.DepTime)*(2.*np.pi/2400))
    X['CRSDepTime_sin'] = np.sin((X.CRSDepTime)*(2.*np.pi/2359))
    X['CRSDepTime_cos'] = np.cos((X.CRSDepTime)*(2.*np.pi/2359))
    X['ArrTime_sin'] = np.sin((X.ArrTime)*(2.*np.pi/2400))
    X['ArrTime_cos'] = np.cos((X.ArrTime)*(2.*np.pi/2400))
    X['CRSArrTime_sin'] = np.sin((X.CRSArrTime)*(2.*np.pi/2400))
    X['CRSArrTime_cos'] = np.cos((X.CRSArrTime)*(2.*np.pi/2400))    
    X.drop(columns=cyclical, inplace=True)
    
    return X

In [151]:
class NominalEncoding(TransformerMixin, BaseEstimator):

    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        for column in nominal:
            mapping = X[column].value_counts() / len(X)
            X[column] = X[column].map(mapping)
        return X

In [176]:
class LogTransformer(TransformerMixin, BaseEstimator):

    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        
        #LOG
        X = pd.DataFrame(data=np.ma.log(X.values).filled(0), columns=numerical)
        
        #ESTANDARIZACION
        scaler = StandardScaler()
        X = scaler.fit_transform(X)
        return X

In [182]:
column_enc = ColumnTransformer(
                    [('nominal_encoding', NominalEncoding(), nominal), 
                    ('log_transformation', LogTransformer(), numerical)])

Definim el pipeline

APLICAR PREPROCESAMIENTO, LUEGO EL TRAIN TEST SPLIT, Y LUEGO EL PIPELINE

In [188]:
pipe = Pipeline([('encoding', column_enc), ('model', RandomForestRegressor(n_jobs=-1))])

Definim els paràmetres pel GridSearch

In [39]:
param_grid = dict(
                n_estimators = [50,100,200],
                max_depth = [10, 20, 50],
                max_features = ['auto', 'sqrt']
    )

Dividim entre target i features, i entre train i test.

In [180]:
y = delayed_flights.ArrDelay
X = delayed_flights.drop(columns='ArrDelay')

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.5,
                                        random_state = 0, shuffle = True)

In [157]:
#Separació segons tipus de variable
all_cols = X_train.columns
cyclical = ['Month', 'DayofMonth', 'DayOfWeek', 'DepTime', 'CRSDepTime', 'ArrTime', 'CRSArrTime']
nominal = ['UniqueCarrier', 'FlightNum', 'TailNum', 'Origin', 'Dest', 'CancellationCode']
numerical = ['ActualElapsedTime', 'CRSElapsedTime', 'AirTime', 'DepDelay', 'Distance', 'TaxiIn', 'TaxiOut', 'CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']

In [187]:
y_train.isna().sum()

4176

In [184]:
pipe.fit_transform(X_train, y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
search = HalvingGridSearchCV(pipe, param_grid, random_state=0, cv=3, verbose=1).fit(X, y)
search

- Exercici 2

Agafa un text en anglès que vulguis, i calcula'n la freqüència de les paraules

# Nivell 2

- Exercici 1

Treu les stopwords i realitza stemming al teu conjunt de dades.

# Nivell 3

- Exercici 1

Realitza sentiment analysis al teu conjunt de dades.